#### Only need to run install/authenticate once per Notebook (I think)

In [1]:
# !pip install earthengine-api

In [2]:
# !earthengine authenticate

In [3]:
# # Import the Earth Engine API and initialize it.
# import ee
# ee.Initialize()

# Test the earthengine command by getting help on upload.
# !earthengine upload image -h

From: GEE_downloadPRISM800_by_LandCover.py

In [4]:
import ee
ee.Initialize()
from geopandas import GeoDataFrame
from shapely.geometry import shape
# import matplotlib.pyplot as plt
import ee.mapclient
import pandas as pd

Get NLCD

In [5]:
# Imports NLCD Land Cover Data
nlcd2011 = ee.Image('USGS/NLCD/NLCD2011')

Get points

In [6]:
df_coords = pd.read_csv('C:/Users/Jim/GitHub/prisons/code/R/exports/prison_ctrs_groups_latlon.csv', 
                        compression=None, header=0, sep=',', quotechar='"', low_memory=False, index_col=False)

df_coords.head()


,STATE,group,X,Y
0,AK,216,-151.529276,59.649347
1,AK,256,-135.333568,57.052658
2,AK,323,-132.377690,56.469473
3,AK,1399,-162.592879,66.896473
4,AK,2352,-152.373083,57.804570


In [7]:
df_coords_sub1 = df_coords[df_coords.STATE == "VT"]
df_coords_sub1

,STATE,group,X,Y
4096,VT,382,-72.990790,43.609690
4097,VT,383,-73.205638,44.446527
4098,VT,390,-72.027951,44.395525
4099,VT,392,-72.190936,44.906313
4100,VT,393,-73.132266,44.850090
4101,VT,2646,-73.080646,44.811123
4102,VT,2648,-72.445631,43.988301
4103,VT,4282,-72.438051,43.269814


In [8]:
array_sub1 = df_coords_sub1.to_numpy()
array_sub1

array([['VT', 382, -72.99078972571981, 43.609690292624855],
       ['VT', 383, -73.20563789558545, 44.446527100239074],
       ['VT', 390, -72.02795108949539, 44.395525260118774],
       ['VT', 392, -72.19093592889806, 44.90631302571161],
       ['VT', 393, -73.1322655013431, 44.85009049694831],
       ['VT', 2646, -73.08064623120555, 44.8111227879875],
       ['VT', 2648, -72.44563089625301, 43.988300959423924],
       ['VT', 4282, -72.43805111553525, 43.269813822491294]], dtype=object)

In [9]:
valuesList = array_sub1.tolist()
valuesList

[['VT', 382, -72.99078972571981, 43.609690292624855],
 ['VT', 383, -73.20563789558545, 44.446527100239074],
 ['VT', 390, -72.02795108949539, 44.395525260118774],
 ['VT', 392, -72.19093592889806, 44.90631302571161],
 ['VT', 393, -73.1322655013431, 44.85009049694831],
 ['VT', 2646, -73.08064623120555, 44.8111227879875],
 ['VT', 2648, -72.44563089625301, 43.988300959423924],
 ['VT', 4282, -72.43805111553525, 43.269813822491294]]

In [10]:
valuesList = ee.List(valuesList) # // Cast valuesList

In [11]:
valuesList.getInfo()

[['VT', 382, -72.99078972571981, 43.609690292624855],
 ['VT', 383, -73.20563789558545, 44.446527100239074],
 ['VT', 390, -72.02795108949539, 44.395525260118774],
 ['VT', 392, -72.19093592889806, 44.90631302571161],
 ['VT', 393, -73.1322655013431, 44.85009049694831],
 ['VT', 2646, -73.08064623120555, 44.8111227879875],
 ['VT', 2648, -72.44563089625301, 43.988300959423924],
 ['VT', 4282, -72.43805111553525, 43.269813822491294]]

In [12]:
def list2fc(l):
    l1 = ee.List(l)
    geom = ee.Geometry.Point([l1.get(2), l1.get(3)])
    return ee.Feature(geom, {'state': l1.get(0), 'group': l1.get(1), 'longitude': l1.get(2), 'latitude': l1.get(3)})

In [13]:
myFeatures = ee.FeatureCollection(valuesList.map(list2fc))

In [14]:
myFeatures

In [15]:
myFeatures.getInfo()

{'type': 'FeatureCollection',
 'columns': {'group': 'Long',
  'latitude': 'Float',
  'longitude': 'Float',
  'state': 'String',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-72.99078972571981, 43.609690292624855]},
   'id': '0',
   'properties': {'group': 382,
    'latitude': 43.609690292624855,
    'longitude': -72.99078972571981,
    'state': 'VT'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.20563789558545, 44.446527100239074]},
   'id': '1',
   'properties': {'group': 383,
    'latitude': 44.446527100239074,
    'longitude': -73.20563789558545,
    'state': 'VT'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-72.02795108949539, 44.395525260118774]},
   'id': '2',
   'properties': {'group': 390,
    'latitude': 44.395525260118774,
    'longitude': -72.02795108949539,
    'state': 'VT'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    '

In [16]:
def bufferPoly(feature):
    return feature.buffer(10000)

In [17]:
Buffers = myFeatures.map(bufferPoly)

In [18]:
Buffers

In [19]:
Buffers.getInfo()

{'type': 'FeatureCollection',
 'columns': {'group': 'Long',
  'latitude': 'Float',
  'longitude': 'Float',
  'state': 'String',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-72.9907897257198, 43.69967895213967],
      [-73.02555394595963, 43.6960921411962],
      [-73.05753928763954, 43.685618446434624],
      [-73.08419172507905, 43.66909487193299],
      [-73.10338788633494, 43.647841027074385],
      [-73.11360463576793, 43.62355272819286],
      [-73.11403877191866, 43.598165759106095],
      [-73.10466780894083, 43.57370102673338],
      [-73.0862479986945, 43.5521036706268],
      [-73.06025092715474, 43.535088937265144],
      [-73.02874472192529, 43.52400686387398],
      [-72.99422982858427, 43.5197361634196],
      [-72.95944229499794, 43.5226153402811],
      [-72.92713950137811, 43.53241619271879],
      [-72.89988430624359, 43.5483616737513],
      [-72.87984366995174, 43.56918678217863],
      [-72

In [20]:
# // Clip the image to the polygon geometry
nlcd2011 = nlcd2011.clip(Buffers)

In [21]:
# // Extract the landcover band
landcover = nlcd2011.select('landcover')
landcover


Can't make a map here in Python apparently, but check out this: https://stackoverflow.com/questions/46960328/workaround-for-google-earth-engine-python-api-and-no-support-for-ee-mapclient


Follow this: https://stackoverflow.com/questions/47633088/get-results-in-an-earth-engine-python-script

In [22]:
# //Print out the frequency of landcover occurrence for each county
frequency = landcover.reduceRegions(collection = Buffers, reducer = ee.Reducer.frequencyHistogram(), scale = 30)
 
# // Prints Feature collection
# frequency = frequency.select(['histogram'])
# frequency.getInfo()


#### NEXT, need to get processed to a csv


#### Trying to get a feature collection from a dataframe, see this

https://gis.stackexchange.com/questions/278607/earth-engine-convert-list-with-coordinates-and-values-into-a-feature-collection

#### Better yet, do something like this first:

https://gis.stackexchange.com/questions/303138/google-earth-engine-remove-geometry-fields-from-featurecollection-for-export-t

In [128]:
# // simply remove geometry
def extract1(feat):
    feat = feat.setGeometry(None)
    return feat

hist = frequency.map(extract1)

data1 = hist.getInfo()

Now work on this, maybe https://pypi.org/project/flatten-dict/

In [130]:
type(data1)

dict

In [138]:
wanted_keys = ['features'] # The keys you want
data2 = dict((k, data1[k]) for k in wanted_keys if k in data1)
data2 = data2.get('features')

In [139]:
type(data2)

list

In [68]:
data3 = data2.get('features')

In [88]:
type(data3)

list

In [140]:
data2

[{'type': 'Feature',
  'geometry': None,
  'id': '0',
  'properties': {'group': 382,
   'histogram': {'11': 1001.0901960784314,
    '21': 15896.325490196086,
    '22': 13382.090196078438,
    '23': 12412.137254901962,
    '24': 3090.788235294118,
    '31': 846.0,
    '41': 138979.9137254896,
    '42': 48878.1764705882,
    '43': 41500.52941176468,
    '52': 2435.6431372549023,
    '71': 4162.470588235295,
    '81': 31762.976470588255,
    '82': 7369.286274509804,
    '90': 20585.729411764725,
    '95': 3247.5137254901956},
   'latitude': 43.609690292624855,
   'longitude': -72.99078972571981,
   'state': 'VT'}},
 {'type': 'Feature',
  'geometry': None,
  'id': '1',
  'properties': {'group': 383,
   'histogram': {'11': 108111.09019607835,
    '21': 29022.231372549028,
    '22': 27142.61176470589,
    '23': 24710.262745098044,
    '24': 11158.592156862745,
    '31': 721.0,
    '41': 36318.149019607874,
    '42': 10282.188235294121,
    '43': 15253.023529411776,
    '52': 563.635294117647

In [141]:
finalMap = []
for d in data2:
    d = d.get('properties')
    finalMap.append(d)
finalMap

[{'group': 382,
  'histogram': {'11': 1001.0901960784314,
   '21': 15896.325490196086,
   '22': 13382.090196078438,
   '23': 12412.137254901962,
   '24': 3090.788235294118,
   '31': 846.0,
   '41': 138979.9137254896,
   '42': 48878.1764705882,
   '43': 41500.52941176468,
   '52': 2435.6431372549023,
   '71': 4162.470588235295,
   '81': 31762.976470588255,
   '82': 7369.286274509804,
   '90': 20585.729411764725,
   '95': 3247.5137254901956},
  'latitude': 43.609690292624855,
  'longitude': -72.99078972571981,
  'state': 'VT'},
 {'group': 383,
  'histogram': {'11': 108111.09019607835,
   '21': 29022.231372549028,
   '22': 27142.61176470589,
   '23': 24710.262745098044,
   '24': 11158.592156862745,
   '31': 721.0,
   '41': 36318.149019607874,
   '42': 10282.188235294121,
   '43': 15253.023529411776,
   '52': 563.635294117647,
   '71': 332.4666666666667,
   '81': 55516.305882352885,
   '82': 5921.858823529412,
   '90': 15350.41960784314,
   '95': 5212.364705882353},
  'latitude': 44.446527

https://stackoverflow.com/questions/14692690/access-nested-dictionary-items-via-a-list-of-keys

getting a little closer